In [1]:
import pandas as pd 
from datetime import datetime
import matplotlib.pyplot as plt
import utils
import numpy as np
import importlib 
from tqdm import tqdm

importlib.reload(utils)


pygame 2.6.0 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


<module 'utils' from 'c:\\Users\\UFMG\\Periodos\\Periodo_XIV\\TCC\\Projeto\\Códigos\\tcc_ovitraps\\utils.py'>

In [75]:
data = pd.read_csv('./data/final_data.csv',parse_dates=['dtcol'])

In [3]:
lags = 3
n_traps = 3

### Treatment 
TODO: move to file

In [98]:
groups = data.groupby(['latitude','longitude'])
rows_to_update = []

for name, group in groups:
    if group['narmad'].nunique() > 1:  # More than one unique value in C
        rows_to_update.extend(group.index.tolist())  # Add row indices to the list
bad_data = data.loc[rows_to_update]

print('Traps with the same lat_long ',bad_data['narmad'].value_counts())
print('\nDates with traps with the same lat_long ',bad_data.groupby(['dtcol'])['novos'].nunique().sort_values(ascending=False).head(10))

Traps with the same lat_long  narmad
902026    305
902025    301
908064    300
908059    300
908032    300
902031    299
908027    299
908207    290
902208    229
908242    228
902206    226
908262     15
908301     15
Name: count, dtype: int64

Dates with traps with the same lat_long  dtcol
2023-10-23    7
2023-12-04    6
2023-11-20    6
2023-04-10    6
2022-04-25    6
2022-01-31    6
2023-12-18    5
2023-05-09    5
2019-05-06    5
2023-01-30    5
Name: novos, dtype: int64


In [72]:
# This cell creates a new dataframe changing the number of the traps that are in the same position
# TODO move to data exploration of preprocessing
same_position_armad = data.groupby(['latitude','longitude'])['narmad'].nunique()
print(same_position_armad.value_counts()) #number of traps in the same position

#data['narmad'] = data.groupby(['latitude','longitude'])['narmad'].transform('first') #group by position and take the first trap number

narmad
1    1760
2       5
3       1
Name: count, dtype: int64


### Distance Matrix

In [13]:
position_matrix = data[['latitude','longitude','narmad']].drop_duplicates().dropna().reset_index(drop=True)

try: 
    distance_matrix = pd.read_csv('./results/distance_matrix.csv',index_col=0)
    distance_matrix.columns = distance_matrix.columns.astype(float)
    distance_matrix.set_index(distance_matrix.columns,inplace=True)


except:
    distance_matrix = utils.create_distance_matrix(position_matrix)
    distance_matrix.to_csv('./results/distance_matrix.csv')

distance_matrix_np = distance_matrix.to_numpy()


### Week trap matrix

In [68]:
week_trap_df = data.pivot(index=['ano','semepi'],columns='narmad',values='novos')
new_index = pd.MultiIndex.from_product([week_trap_df.index.levels[0], range(101,153)]) # introduce weeks 51 and 52
new_index = new_index[(new_index <= week_trap_df.iloc[-1].name) & (new_index >= week_trap_df.iloc[0].name)] #remove indexes that are greater than the last sample or smaller than the first one
week_trap_df = week_trap_df.reindex(new_index) # [week,trap] - > novos


ValueError: Index contains duplicate entries, cannot reshape

### Lagged Matrix

In [15]:
list_lagged_df = []
for i in range(1,2*lags+1):
    list_lagged_df.append(week_trap_df.shift(i).values)
lagged_matrix = np.stack(list_lagged_df, axis=0) # numpy array with the lagged values. [lag x week x trap] -> novos. Obs.: consider 2*lags due to the biweekly sampling rate


### NaN Count Matrix

In [16]:
nan_count_matrix = np.sum(np.isnan(lagged_matrix), axis=0) # [week x trap] -> number of nans in the lagged matrix

for i in range(2*lags):
    print("Number of samples with",i+1,"valid values: ",np.sum(nan_count_matrix==i+1))

Number of samples with 1 valid values:  0
Number of samples with 2 valid values:  0
Number of samples with 3 valid values:  820386
Number of samples with 4 valid values:  295751
Number of samples with 5 valid values:  33236
Number of samples with 6 valid values:  204019


### Info Matrix

In [17]:
info_df = data[['ano','semepi','nplaca','dtcol','novos','narmad','latitude','longitude']]

### Lagged days matrix

In [18]:
day_df = data.pivot(index=['ano','semepi'],columns='narmad',values='dtcol')
new_index = pd.MultiIndex.from_product([day_df.index.levels[0], range(101,153)]) # introduce weeks 51 and 52
new_index = new_index[(new_index <= day_df.iloc[-1].name) & (new_index >= day_df.iloc[0].name)] #remove indexes that are greater than the last sample or smaller than the first one
day_df = day_df.reindex(new_index) # [week,trap] - > dtcol
day_df = day_df.map(lambda x: x.toordinal() if pd.notnull(x) else np.nan) # convert to ordinal so we can calculate the difference between two dates

list_lagged_days = []
for i in range(1,2*lags+1):
    list_lagged_days.append(day_df.shift(i).values)
lagged_days = np.stack(list_lagged_days, axis=0) # numpy array with the lagged values. [lag x week x trap] -> ordinal daus. Obs.: consider 2*lags due to the biweekly sampling rate




### Useful dicts

In [19]:
trap_index_dict = {trap: index for index,trap in enumerate(distance_matrix.columns)}                                           # trap: index 
yearweek_index_dict = {(year,week): index for index,(year,week) in enumerate(week_trap_df.index)}                           # (year,week): index
nplaca_week_dict = {nplaca: (year, week) for nplaca,week,year in zip(info_df['nplaca'],info_df['semepi'],info_df['ano'])}   # nplaca: (year,week)
nplaca_index_dict = {nplaca: yearweek_index_dict[(year, week)] for nplaca,week,year in zip(info_df['nplaca'],info_df['semepi'],info_df['ano'])}   # nplaca: week index 


### Final Dataframe

In [20]:
for trap in distance_matrix.columns:
    matching_placas = info_df[info_df['narmad'] == trap]['nplaca']
    trap_index = trap_index_dict[trap]
    sorted_distance_indexes = np.argsort(distance_matrix_np[trap_index])
    assert sorted_distance_indexes[0] == trap_index
    

    for placa in matching_placas: 
        pass

In [21]:
utils.row_with_value(info_df,'narmad',trap)

,ano,semepi,nplaca,dtcol,novos,narmad,latitude,longitude
517783,2024.0,120.0,90524387.0,2024-05-22,6.0,909195.0,-19.7934,-43.97873
517784,2024.0,110.0,90524373.0,2024-03-13,49.0,909195.0,-19.7934,-43.97873
517785,2024.0,122.0,90524380.0,2024-06-05,0.0,909195.0,-19.7934,-43.97873
517786,2024.0,118.0,90524383.0,2024-05-06,39.0,909195.0,-19.7934,-43.97873
517787,2024.0,130.0,90524386.0,2024-07-29,5.0,909195.0,-19.7934,-43.97873
517788,2024.0,132.0,90524376.0,2024-08-14,24.0,909195.0,-19.7934,-43.97873
517789,2024.0,134.0,90524378.0,2024-08-28,0.0,909195.0,-19.7934,-43.97873
517790,2024.0,116.0,90524381.0,2024-04-25,0.0,909195.0,-19.7934,-43.97873
517791,2024.0,112.0,90524379.0,2024-03-27,0.0,909195.0,-19.7934,-43.97873
517792,2024.0,102.0,90524377.0,2024-01-17,238.0,909195.0,-19.7934,-43.97873


In [22]:
info_df.groupby('narmad').min()

,ano,semepi,nplaca,dtcol,novos,latitude,longitude
narmad,,,,,,,
901001,2012.0,101,90000001,2012-05-14,0.0,-19.974521,-44.057499
901002,2012.0,101,90000285,2012-09-03,0.0,-19.984561,-44.053943
901003,2012.0,101,90000568,2012-09-03,0.0,-19.977669,-44.053692
901004,2012.0,101,90000851,2012-09-03,0.0,-19.974816,-44.061480
901005,2012.0,101,90001134,2012-09-03,0.0,-19.977715,-44.057558
...,...,...,...,...,...,...,...
909190,2023.0,102,90524298,2023-11-21,0.0,-19.796700,-43.947889
909191,2023.0,102,90524317,2023-11-21,0.0,-19.786108,-43.971220
909192,2023.0,102,90524336,2023-12-06,0.0,-19.782779,-43.975511


In [23]:
distance_matrix.iloc[242].name

np.float64(902025.0)

In [24]:
sorted_distance_indexes[:10]

array([1765, 1636, 1756, 1720, 1645, 1639, 1646, 1734, 1739, 1637])